# Optimizing naive Gillespie code using Cython

In [1]:
import numpy as np
import cython
Na = 6.023e23  # Avogadro's constant

In [2]:
%load_ext Cython

In [3]:
# Original roulette selection
def roulette_selection(prop_list, Xt):
    """Perform roulette selection on the list of propensities"""
    prop = np.copy(prop_list)
    prop0 = np.sum(prop)  # Sum of propensities
    if prop0 == 0:
        if np.sum(Xt) == 0:
            status = 3
            return [-1, status]
        else:
            status = -2
            return [-1, status]
    prop = prop / prop0  # Normalize propensities to be < 1
    # Concatenate 0 to list of probabilities
    probs = [0]+list(np.cumsum(prop))
    r1 = np.random.rand() # Roll the wheel
    # Identify where it lands and update that reaction
    for ind1 in range(len(probs)):
        if r1 <= probs[ind1]:
            choice = ind1 - 1
            break
    return [choice, 0]

In [4]:
%%cython -a

import cython
import numpy as np


@cython.boundscheck(False)
@cython.wraparound(False)
cpdef (int, int) cy_roulette_selection(double[:] prop_list, long[:] Xt):
    """Perform roulette selection on the list of propensities"""
    cdef int prop0 = np.sum(prop_list)  # Sum of propensities
    cdef int status
    if prop0 == 0:
        if np.sum(Xt) == 0:
            status = 3
            return -1, status
        else:
            status = -2
            return -1, status
    cdef double[:] prop = np.divide(prop_list, prop0)  # Normalize propensities to be < 1
    # Concatenate 0 to list of probabilities
    probs = [0] + list(np.cumsum(prop))
    cdef float r1 = np.random.random() # Roll the wheel
    # Identify where it lands and update that reaction
    cdef int ind1
    for ind1 in range(len(probs)):
        if r1 <= probs[ind1]:
            choice = ind1 - 1
            break
    return choice, 0

In [5]:
# Original get_kstoc function
def get_kstoc(k_det, V_r, volume = 1.0):
    """Compute k_stoc from k_det"""
    Na = 6.023e23  # Avogadro's constant
    nr = V_r.shape[0]  # Number of reactions
    orders = np.sum(V_r, 1)  # Order of rxn = number of reactants
    k_stoc = np.copy(k_det)
    for ind in range(nr):
        # If highest order is 3
        if np.max(V_r[ind, :]) == 3:
            k_stoc[ind] = k_det[ind] * 6 / np.power(Na * volume, 2)
        elif np.max(V_r[ind, :]) == 2:  # Highest order is 2
            k_stoc[ind] = k_det[ind] * 2 / np.power(Na * volume, orders[ind]-1)
        else:
            k_stoc[ind] = k_det[ind] / np.power(Na * volume, orders[ind]-1)
    return k_stoc

In [6]:
%%cython -a

import cython
import numpy as np

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef double[:] cy_get_kstoc(double[:] k_det, long[:, :] V_r, float volume = 1.0):
    """Compute k_stoc from k_det"""
    cdef double Na = 6.023e23  # Avogadro's constant
    cdef int nr = V_r.shape[0]  # Number of reactions
    cdef long[:] orders = np.sum(V_r, 1)  # Order of rxn = number of reactants
    k_stoc = np.zeros_like(k_det)
    cdef int ind
    for ind in range(nr):
        # If highest order is 3
        if np.max(V_r[ind, :]) == 3:
            k_stoc[ind] = k_det[ind] * 6 / np.power(Na * volume, 2)
        elif np.max(V_r[ind, :]) == 2:  # Highest order is 2
            k_stoc[ind] = k_det[ind] * 2 / np.power(Na * volume, orders[ind] - 1)
        else:
            k_stoc[ind] = k_det[ind] / np.power(Na * volume, orders[ind] - 1)
    return k_stoc

In [7]:
# Original direct_native function
def direct_naive( V_r, V_p, X0, k_det, max_t=1.0, max_iter=100, volume=1.0, seed = 0):
    """Naive implementation of the Direct method"""
    ite = 1  # Iteration counter
    t = 0  # Time in seconds
    nr = V_r.shape[0]  # Number of reactions
    ns = V_r.shape[1]  # Number of species

    if (nr != V_p.shape[0]) or (ns != V_p.shape[1]):
        raise ValueError('V_r and V_p should be the same shape.')

    if (nr != k_det.shape[0]):
        raise ValueError('Number of elements in k_det must equal\
         number of rows in V_r.')

    if np.any(V_r < 0):
        raise ValueError('V_r cannot have negative elements.')

    if np.any(V_p < 0):
        raise ValueError('V_p cannot have negative elements.')

    if np.any(X0 < 0):
        raise ValueError('Initial numbers in X0 can\'t be negative.')

    if np.any(k_det < 0):
        neg_indices = np.where(k_det < 0)[0]
        raise ValueError('Rate constant(s) at position(s) ' + str(neg_indices) + ' are negative.')

    V = V_p - V_r  # nr x ns
    Xt = np.copy(X0)  # Number of species at time t
    Xtemp = np.zeros(nr)  # Temporary X for updating
    kstoc = np.zeros(nr)  # Stochastic rate constants
    orders = np.sum(V_r, 1)  # Order of rxn = number of reactants
    status = 0
    np.random.seed(seed=seed)  # Set the seed

    if np.max(orders) > 3:
        raise ValueError('Order greater than 3 detected.')

    # Determine kstoc from kdet and the highest order or reactions
    kstoc = get_kstoc(k_det, V_r, volume)
    prop = np.copy(kstoc)  # Vector of propensities

    while ite < max_iter:
        # Calculate propensities
        for ind1 in range(nr):
            for ind2 in range(ns):
                # prop = kstoc * product of (number raised to order)
                prop[ind1] *= np.power(Xt[ind2], V_r[ind1, ind2])
        # Roulette wheel
        [choice, status] = roulette_selection(prop, Xt)
        if status == 0:
            Xtemp = Xt + V[choice, :]
        else:
            return t, Xt, status

        # If negative species produced, reject step
        if np.min(Xtemp) < 0:
            continue
        # Update Xt and t
        else:
            Xt = Xtemp
            r2 = np.random.rand()
            t += 1 / np.sum(prop) * np.log(1 / r2)
            if t > max_t:
                status = 2
                print("Reached maximum time (t = )", t)
                return t, Xt, status
        prop = np.copy(kstoc)
        ite += 1
    status = 1
    return t, Xt, status


In [8]:
%%cython -a

import cython
import numpy as np
from __main__ import cy_get_kstoc, cy_roulette_selection


@cython.boundscheck(False)
@cython.wraparound(False)
cpdef cy_direct_naive(
    long[:, :] V_r, 
    long[:, :] V_p,
    long[:] X0,
    double[:] k_det,
    float max_t = 1.0,
    long max_iter = 100,
    float volume = 1.0,
    int seed = 0
):
    """Naive implementation of the Direct method"""
    cdef int ite = 1  # Iteration counter
    cdef double t = 0.0  # Time in seconds
    cdef int nr = V_r.shape[0]  # Number of reactions
    cdef int ns = V_r.shape[1]  # Number of species

    if (nr != V_p.shape[0]) or (ns != V_p.shape[1]):
        raise ValueError('V_r and V_p should be the same shape.')

    if (nr != k_det.shape[0]):
        raise ValueError('Number of elements in k_det must equal\
         number of rows in V_r.')

    if np.any(np.less(V_r, 0)):
        raise ValueError('V_r cannot have negative elements.')

    if np.any(np.less(V_p, 0)):
        raise ValueError('V_p cannot have negative elements.')

    if np.any(np.less(X0, 0)):
        raise ValueError('Initial numbers in X0 can\'t be negative.')

    if np.any(np.less(k_det, 0)):
        neg_indices = np.where(np.less(k_det, 0))[0]
        raise ValueError('Rate constant(s) at position(s) ' + str(neg_indices) + ' are negative.')

    V = np.subtract(V_p, V_r)  # nr x ns
    cdef long[:] Xt = np.copy(X0)  # Number of species at time t
    cdef long[:] Xtemp = np.zeros(nr, dtype=int)  # Temporary X for updating
    cdef double[:] kstoc = np.zeros(nr)  # Stochastic rate constants
    cdef long[:] orders = np.sum(V_r, 1)  # Order of rxn = number of reactants
    cdef int status = 0
    np.random.seed(seed=seed)  # Set the seed

    if np.max(orders) > 3:
        raise ValueError('Order greater than 3 detected.')

    # Determine kstoc from kdet and the highest order or reactions
    kstoc = cy_get_kstoc(k_det, V_r, volume)
    prop = np.copy(kstoc)  # Vector of propensities

    while ite < max_iter:
        # Calculate propensities
        for ind1 in range(nr):
            for ind2 in range(ns):
                # prop = kstoc * product of (number raised to order)
                prop[ind1] *= np.power(Xt[ind2], V_r[ind1, ind2])
        # Roulette wheel
        [choice, status] = cy_roulette_selection(prop, Xt)
        if status == 0:
            Xtemp = Xt + V[choice, :]
        else:
            return t, Xt, status

        # If negative species produced, reject step
        if np.min(Xtemp) < 0:
            continue
        # Update Xt and t
        else:
            Xt = Xtemp
            r2 = np.random.rand()
            t += 1 / np.sum(prop) * np.log(1 / r2)
            if t > max_t:
                status = 2
                print("Reached maximum time (t = )", t)
                return t, Xt, status
        prop = np.copy(kstoc)
        ite += 1
    status = 1
    return t, Xt, status


## Profiling and Timing

In [9]:
V_r = np.array([[1, 0, 0, 0], [0, 1, 0, 1], [1, 0, 0, 0]])
V_p = np.array([[0, 1, 0, 0], [0, 2, 0, 0], [0, 0, 1, 0]])
k = np.array([1, 0.01 * Na, 1])
X0 = np.array([1, 0, 0, 10])
count_excitation = 0
n_runs = 1000
deviation_tolerance = 0.05

In [10]:
%%timeit
count_excitation = 0
# python version
for ind in range(n_runs):
    [_, Xt, _] = direct_naive(V_r, V_p, X0, k, max_t=150, max_iter=1000, seed=ind)
    assert np.all(Xt - np.array([0, 11, 0, 0]) == 0) or np.all(Xt - np.array([0, 0, 1, 10]) == 0)
    if np.all(Xt - np.array([0, 11, 0, 0]) == 0):
        count_excitation += 1

388 ms ± 9.48 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%%timeit
# cython version
for ind in range(n_runs):
    [_, Xt, _] = cy_direct_naive(V_r, V_p, X0, k, max_t=150, max_iter=1000, seed=ind)
    assert np.all(Xt - np.array([0, 1, 0, 10]) == 0) or np.all(Xt - np.array([0, 0, 1, 10]) == 0)
    if np.all(Xt - np.array([0, 11, 0, 0]) == 0):
        count_excitation += 1

224 ms ± 2.96 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
